# HMM GMM

We use 3-state mono-phone HMMs to construct this recognizer. The emission probability of every state is modeled by a GMM. Say we have F mono-phones (F = 39 in our English lexicon), and a G-mixture GMM for each mono-phone state. Thus the GMM-HMM has 3FG mixture components in total. Compared to GMM-UBM, these mixtures are better separated in the phoneme space.

MFCC:  MFCCs  are  extracted  from  16kHz  utterance  with  40 filter-banks distributed between 0 and 8kHz. Static 19-dimensional coefficients plus energy and their delta and delta-delta form a 60-dimensional vector. CMVN is applied per utterance.

Hmm, deepspeech ma 26 MFCC features i zdaje się, że usuwa co drugi, ale to w związku z jakąś sztuczką uczenia RNN. Mają tak: features = mfcc(audio, samplerate=fs, numcep=numcep) gdzie mfcc jest z psf jak u nas xD a numcep=26 domyślnie, a fs=16000 domyślnie

Hmm, inny papier: In our HMM-based method, a phoneme recognizer is firsttrained with 3-state, GMM-based, mono-phone HMMs. Thisrecognizer is the same as in speech recognition. LetFbe thetotal number of mono-phones (i.e. 39),S=3Fbe the numberof all states,Gthe number of Gaussian components per state,andC=SGthe number of all individual Gaussians, and let(s, g)denote Gaussian componentgin states.
3-state - what are you?

Given a transcription, a graph of HMM is composed.

The  Viterbi  and  forward-backward  (FB)  algorithms  are  twomeans to align frames to states and mixtures.

Speaker adaption is the same with Eq. 4 except mixtures here are phonetic dependent.

During the test phase,  the Viterbi-based log-likelihood ratio is expressed as:
sum_t log P(x_t | model_user,qt) - log P(x_t | model_ubm,qt)

HMM: To generate the alignment for the HMM-based modeling,we use MFCCs to train the HMM. 39 mono-phones plus a silencemodel are used,  each of which contains 3 states.  To model thecomplexity of silence, a GMM with 16 mixtures is used for every silence state, while other states are all modeled by 8 Gaussians,resulting 984 Gaussians in total. This HMM is further extended toa triphone system and remains 2142 senones.  The transcriptionsfor  DNN  training  is  generated  by  the  senone  alignment.   OnlyMFCCs are used for HMM training and alignment.

GMM-HMM and i-vector/HMM: The GMM of every state is re-estimated using the HMM alignments and different speaker fea-tures.   The total number of mixtures in our model is 984.   Thedimension of i-vector is again set to 600.  Viterbi and FB align-ments are both investigated

* 39 x 3 states

## MAP - Książka Rabinera:

MAP adaptation of speaker-independent model to a speaker of parameters is obtained by solving 

$$\frac{d}{d\lambda} P(\lambda | O) = 0$$

where

$$P(\lambda | O) = \frac{P(O | \lambda)P(\lambda)}{P(O)}$$

If we have a good prior estimated from a lot of data, then we can use it to get a model for a speaker for which data is scarce.

Conjugate priors for a random vector is defined as the prior distribution for the parameters of the parameters of the probaiblity density function of the random vector, such that the posterior distribution $P(\lambda | O)$ and the prior distribution $P(\lambda)$ belong to the same distribution family for any sample observations $O$.
For example, it is well known that the conjugate prior for the mean of a Gaussian density is also a Gaussian density.

$$\bar{u_{MAP}} = \frac{n \tau^2}{\sigma^2 + n \tau^2} \bar{o} + \frac{\sigma^2}{\sigma^2 + n \tau^2} \rho$$

where $\bar{o}$ - sample mean of new data, $n$ - number of training observations, $\tau^2$ - variance of prior, $\rho$ - mean of prior distribution

How do we estimate $\rho$ and $\tau^2$ of prior? From a collection of speaker-dependent models (and $c_m$ is a weight of a model) or from a speaker-independent model with mixture of distributions in each state ($c_m$ is a weight of a mixture component)

$\rho = \sum_{m=1}^M c_m \rho_m$

$\tau^2 = \sum_{m=1}^M c_m (\rho_m - \rho)$

System do rozpoznawania słowa:

po jednym modelu na słowo, rozpoznając matchujemy z każdym po kolei i bierzemy max prob
model left-to-right zamiast ergodic :OOOO

https://web.stanford.edu/class/cs224s/lectures/

Hmm, Viterbi learning, hard assignment: Bahm ~ o3 
EM learning, soft assignment: Bahm ~ 0*o1 + 0.15*o2 + 0.5*o3 + 0.05*o

Typical training procedure in LVCSR
Generate a forced alignment with existing model
Viterbi decoding with a very constrained prior (the transcript)
Assigns observations to HMM states
Create new observation models from update alignments
Iteratively repeat the above steps, occasionally introducing a more complex observation model or adding more difficult training examples

Viterbi Beam search

Applying FB to speech: Caveats
Network structure of HMM is always created by hand
no algorithm for double-induction of optimal structure and probabilities has been able to beat simple hand-built structures.
Always Bakisnetwork = links go forward in time
Subcase of Bakisnet: beads-on-string net

Assume that the features in the feature vector are uncorrelated
This isn’t true for FFT features, but is true for MFCC features, as we will see later


Initialization: “Flat start”Transition probabilities:set to zero any that you want to be “structurally zero”Set the rest to identical valuesLikelihoods:initialize μand sof each state to global mean and variance of all training data

Embedded TrainingGiven: phoneset, lexicon, transcribed wavefilesBuild a whole sentence HMM for each sentenceInitialize A probsto 0.5, or to zeroInitialize B probsto global mean and varianceRun multiple iterations of Baum WelchDuring each iteration, we compute forward and backward probabilitiesUse them to re-estimate A and BRun Baum-Welch until convergence

Viterbi training rather than Baum-Welch training. Computing the “Viterbi path” over the training data is called “forced alignment”
Because we know which word string to assign to each observation sequence.
We just don’t know the state sequence.
So we use aijto constrain the path to go through the correct words

Modeling phonetic context
The strongest factor affecting phonetic variability is the neighboring phone
Idea: have phone models which are specific to context
Instead of Context-Independent (CI) phonesWe’ll have Context-Dependent (CD) phones


Speech signal is not constantslope of formants, change from stop burst to releaseSo in addition to the cepstralfeaturesNeed to model changes in the cepstralfeatures over time.“delta features”“double delta” (acceleration) features

Typical MFCC featuresWindow size: 25msWindow shift: 10msPre-emphasis coefficient: 0.97MFCC:12 MFCC (mel frequency cepstral coefficients)1 energy feature12 delta MFCC features 12 double-delta MFCC features1 delta energy feature1 double-delta energy featureTotal 39-dimensional features


Acoustic Model AdaptationShift the means and variances of Gaussians to better match the input feature distributionMaximum Likelihood Linear Regression (MLLR)Maximum A Posteriori (MAP) AdaptationFor both speaker adaptation and environment adaptation

MAP Adaptation: u_new = N / (N + a) u_estimated + a / (N + a) u_old, where a is some weight of old

DUŻO BYŁO W lec5

SIL is a phoneme to a recognizerAlways inserted at start and end of utteranceCorrupting silence with bad forced alignments can break recognizer training (silence eats everything)

## Hmm, no dobra, plan działania:

Preprocessing:
wytnij ciszę z dźwięków, voice activity detection 
Wylicz MFCC, 20 stanów + d + dd 

Training:
Dla każdego sentence_id (jest ich 10):
    Utwórz HMMGMM z całego zbioru treningowego sequence_id:
        HMM with 3 states and 8 Gaussian components for each of 39 mono-phones were used for the alignment (resulting in to-tal number of 936 Gaussian components) ?? 
        albo unique(znaki) * 3 stany
        albo znaki * 3 stany trójkątna! (Bakis)
        
Enrollment:
Dla każdego gspeaker_id:
    Dla każdego sentence_id:
        Weź dane (gspeaker_id, sequence_id)
        Naucz HMM taki jak wyżej tylko na nich
        Dokonaj MAP adaptacji
        
Trial:
likelihood HMM_{podany speaker, podane zdanie} - likelihood HMM_{UBM, podane zdanie} ? 

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

import concurrent.futures as cf
import functools as ft
import itertools as it
import json
import math
import operator as op
import os
import re

from IPython.display import display
from ipywidgets import interact, interact_manual, widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pomegranate as pg
from scipy import interpolate, linalg, misc, optimize, spatial, stats
from sklearn import metrics, mixture, cluster, utils

from paprotka.dataset import reddots
from paprotka.feature import cepstral

In [5]:
%autoreload 0

In [6]:
import warnings
warnings.filterwarnings('once')

# Load

In [7]:
root = reddots.get_root()
load_pcm = ft.partial(reddots.load_pcm, root)
load_mfcc = ft.partial(reddots.load_npy, root, 'wac2_mfcc13_ddd_lifter22_hamming')

In [8]:
all_paths = [os.path.join(os.path.basename(root), file) for root, _, files in os.walk(root + '/pcm') 
                                      for file in files 
                                      if file.endswith('.pcm')]
print(len(all_paths), all_paths[0])

15305 f0002/20150224142650384_f0002_14438.pcm


In [9]:
all_mfcc = {path: load_mfcc(path) for path in all_paths}

In [10]:
script = pd.read_csv(root + '/infos/phones.csv', sep=';')
print(script.loc[script['sentence_id'] == 31])

      sentence_id                  content  \
1609           31  My voice is my password   

                                                 phones  
1609  M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D  


In [11]:
def load_sets(eid, tid=None):
    tid = tid if tid else eid
    
    enrollments = reddots.load_enrollments(root + '/ndx/f_part_{}.trn'.format(eid), 
                                           root + '/ndx/m_part_{}.trn'.format(eid))
    trials = reddots.load_trials(root + '/ndx/f_part_{}.ndx'.format(tid), 
                                 root + '/ndx/m_part_{}.ndx'.format(tid))
    
    enrollments = enrollments.merge(script, how='left', on='sentence_id')
    trials = trials.merge(script, how='left', left_on='expected_sentence_id', right_on='sentence_id')
    del trials['sentence_id']
    trials.rename(columns={'content': 'expected_content', 'phones': 'expected_phones'}, inplace=True)
    trials = trials.merge(script, how='left', left_on='trial_sentence_id', right_on='sentence_id')
    del trials['sentence_id']
    trials.rename(columns={'content': 'trial_content', 'phones': 'trial_phones'}, inplace=True)
    
    return enrollments, trials

In [12]:
enrollments_1, trials_1 = load_sets('01')
print('Enrollments', enrollments_1.dtypes, sep='\n')
print('Trials', trials_1.dtypes, sep='\n')

Enrollments
is_male                  bool
pcm_path               object
sentence_id             int16
speaker_id              int16
timestamp      datetime64[ns]
content                object
phones                 object
dtype: object
Trials
correct_sentence                  bool
expected_is_male                  bool
expected_sentence_id             int16
expected_speaker_id              int16
pcm_path                        object
target_person                     bool
trial_is_male                     bool
trial_sentence_id                int16
trial_speaker_id                 int16
trial_timestamp         datetime64[ns]
expected_content                object
expected_phones                 object
trial_content                   object
trial_phones                    object
dtype: object


In [13]:
enrollments_2, trials_2 = load_sets('02')
enrollments_3, trials_3 = load_sets('03')
enrollments_4_td, trials_4_td = load_sets('04_td', '04')

In [14]:
trialed_paths = set(path for trials in (trials_1, trials_2, trials_3, trials_4_td) for path in trials.pcm_path)
untrialed_paths = [path for path in all_paths if path not in trialed_paths]
print(len(trialed_paths), len(untrialed_paths))

10857 4448


In [15]:
shit_lengths = np.array([mfcc.shape[0] for mfcc in all_mfcc.values()])
print(shit_lengths.min(), shit_lengths.mean(), shit_lengths.max(), shit_lengths)

shit_lengths_sorted = np.sort(shit_lengths)

print(shit_lengths_sorted[:40])
print(shit_lengths_sorted[-40:])

1 316.151323097 1219 [342  86 417 ..., 412 181 345]
[ 1  1  1  7  8  9  9  9 11 12 12 12 12 13 23 27 60 60 78 79 80 85 86 87 87
 87 89 89 89 89 89 89 89 89 91 91 92 92 93 93]
[1047 1047 1057 1058 1059 1059 1063 1065 1065 1067 1067 1067 1069 1069 1071
 1076 1078 1078 1078 1082 1086 1088 1088 1090 1181 1181 1181 1181 1181 1181
 1185 1185 1200 1200 1202 1202 1219 1219 1219 1219]


In [16]:
trialed_paths = [path for path in trialed_paths if all_mfcc[path].shape[0] >= 85]
untrialed_paths = [path for path in untrialed_paths if all_mfcc[path].shape[0] >= 85]

In [17]:
# memory blew up
# enrollments_1.mfcc = enrollments_1.apply(lambda row: load_mfcc(row.pcm_path), axis=1)
# trials_1.mfcc = trials_1.apply(lambda row: load_mfcc(row.pcm_path), axis=1)
# enrollments_4_td.mfcc = enrollments_4_td.apply(lambda row: load_mfcc(row.pcm_path), axis=1)

In [18]:
# got_this_back = pg.GeneralMixtureModel(bg_model.distributions, np.exp(bg_model.weights) * bg_model.weights.sum())
# got_this_back = pg.MultivariateGaussianDistribution(means=bg_model.distributions[0].mu, covariance=bg_model.distributions[0].cov)

def with_opened_file(mode='r'):
    def decorator(fun):
        @ft.wraps(fun)
        def wrapped(path, *args, **kwargs):
            with open(path, mode=mode) as opened:
                return fun(opened, *args, **kwargs)
        return wrapped
    return decorator

def write_model(path, model):
    with open(path, 'w') as opened:
        opened.write(model.to_json())
        
def read_mvgaussian(path):
    with open(path, 'r') as opened:
        parsed = json.load(opened)
        means, covariance = parsed['parameters']
        return pg.MultivariateGaussianDistribution(means=means, covariance=covariance)
        
def read_gmm(path):
    with open(path, 'r') as opened:
        parsed = json.load(opened)
        distributions_parsed = parsed['distributions']
        weights_parsed = np.array(parsed['weights'])
        
        distributions = []
        for i, distribution in enumerate(distributions_parsed):
            try:
                distribution = pg.MultivariateGaussianDistribution(*distribution['parameters'])
                distributions.append(distribution)
            except Exception as exception:
                raise IOError('Cannot read mixture at {}'.format(i)) from exception
                        
        weights_exp = np.exp(weights_parsed) * weights_parsed.sum()
        
        return pg.GeneralMixtureModel(distributions=distributions, weights=weights_exp)
    
def write_triples(path, triples):
    for phone, triple in triples.items():
        for i, model in enumerate(triple):
            c = chr(ord('a') + i)
            write_model(path + '/{}-{}'.format(phone, c), model)
            
def read_triples(path, phones):
    triples = {}
    for phone in phones:
        triple = []
        for c in ('a', 'b', 'c'):
            try:
                model = read_gmm(path + '/{}-{}'.format(phone, c))
                triple.append(model)
            except Exception as exception:
                raise IOError('Cannot read model {}-{}'.format(phone, c)) from exception
        triples[phone] = tuple(triple)
    return triples

# Train

In [ ]:
example = enrollments_1[enrollments_1.sentence_id == 31]
example_features = [all_mfcc[path] for path in example['pcm_path']]
example_lengths = [mfcc.shape[0] for mfcc in example_features] 
example_features_stack = np.vstack(example_features)

print(len(example_features), example_features_stack.shape)

In [19]:
untrialed_features = [all_mfcc[path] for path in untrialed_paths]
untrialed_features_stack = np.vstack(untrialed_features)

print(len(untrialed_features), untrialed_features_stack.shape)

4445 (1442782, 39)


## Make 8 and 16 mixtures all sentence models

In [20]:
bg_model = pg.GeneralMixtureModel.from_samples(
    pg.MultivariateGaussianDistribution, n_components=8, X=untrialed_features_stack,
    max_iterations=2000, verbose=True, n_jobs=20
)

[1] Improvement: 2473702.901794061	Time (s): 1.224
[2] Improvement: 633268.254008323	Time (s): 1.141
[3] Improvement: 352966.4639593363	Time (s): 1.166
[4] Improvement: 262230.85166698694	Time (s): 1.137
[5] Improvement: 169611.8659812212	Time (s): 1.165
[6] Improvement: 114529.26232421398	Time (s): 1.118
[7] Improvement: 84511.80233338475	Time (s): 1.199
[8] Improvement: 65141.83384655416	Time (s): 1.157
[9] Improvement: 51947.90546943247	Time (s): 1.177
[10] Improvement: 43309.55319856107	Time (s): 1.199
[11] Improvement: 37513.69882506132	Time (s): 1.12
[12] Improvement: 32930.84429334104	Time (s): 1.095
[13] Improvement: 29065.06633129716	Time (s): 1.079
[14] Improvement: 25951.691959545016	Time (s): 1.126
[15] Improvement: 23539.73283688724	Time (s): 1.164
[16] Improvement: 21588.393727973104	Time (s): 1.177
[17] Improvement: 19916.06888127327	Time (s): 1.196
[18] Improvement: 18325.81861205399	Time (s): 1.053
[19] Improvement: 16639.165491849184	Time (s): 1.185
[20] Improvement: 

[158] Improvement: 1.6510690450668335	Time (s): 1.242
[159] Improvement: 1.4923385977745056	Time (s): 1.189
[160] Improvement: 1.3483123779296875	Time (s): 1.309
[161] Improvement: 1.2177345752716064	Time (s): 1.27
[162] Improvement: 1.0994422435760498	Time (s): 1.337
[163] Improvement: 0.9923547059297562	Time (s): 1.325
[164] Improvement: 0.895471379160881	Time (s): 1.224
[165] Improvement: 0.8078731149435043	Time (s): 1.268
[166] Improvement: 0.7287118434906006	Time (s): 1.38
[167] Improvement: 0.657207801938057	Time (s): 1.192
[168] Improvement: 0.5926496088504791	Time (s): 1.233
[169] Improvement: 0.5343829095363617	Time (s): 1.258
[170] Improvement: 0.4818141609430313	Time (s): 1.343
[171] Improvement: 0.4343985468149185	Time (s): 1.229
[172] Improvement: 0.3916418254375458	Time (s): 1.318
[173] Improvement: 0.3530950993299484	Time (s): 1.236
[174] Improvement: 0.3183496445417404	Time (s): 1.5
[175] Improvement: 0.287035197019577	Time (s): 1.168
[176] Improvement: 0.25881649553775

In [21]:
bg_model.freeze()

In [22]:
silence_model = pg.GeneralMixtureModel.from_samples(
    pg.MultivariateGaussianDistribution, n_components=16, X=untrialed_features_stack,
    max_iterations=4000, verbose=True, n_jobs=20
)

[1] Improvement: 2326001.4778058827	Time (s): 2.332
[2] Improvement: 1048936.2273376584	Time (s): 2.422
[3] Improvement: 503788.152314499	Time (s): 2.523
[4] Improvement: 378417.29803210497	Time (s): 2.198
[5] Improvement: 287822.1143409461	Time (s): 2.189
[6] Improvement: 227266.7865781635	Time (s): 2.281
[7] Improvement: 146496.442282781	Time (s): 2.389
[8] Improvement: 87120.31746590137	Time (s): 2.296
[9] Improvement: 58981.893899172544	Time (s): 2.225
[10] Improvement: 44090.7145973593	Time (s): 2.132
[11] Improvement: 35726.543679997325	Time (s): 2.146
[12] Improvement: 30658.29612390697	Time (s): 2.24
[13] Improvement: 27045.763345122337	Time (s): 2.405
[14] Improvement: 23897.952405139804	Time (s): 2.121
[15] Improvement: 21720.208472058177	Time (s): 2.162
[16] Improvement: 20499.78999952972	Time (s): 2.08
[17] Improvement: 18468.090148806572	Time (s): 2.217
[18] Improvement: 15723.550213769078	Time (s): 2.166
[19] Improvement: 13953.234100610018	Time (s): 2.121
[20] Improvemen

[157] Improvement: 98.31619562208652	Time (s): 2.139
[158] Improvement: 81.87848582863808	Time (s): 2.229
[159] Improvement: 68.34950263798237	Time (s): 2.059
[160] Improvement: 57.1679525077343	Time (s): 2.088
[161] Improvement: 47.888062715530396	Time (s): 2.373
[162] Improvement: 40.17306137084961	Time (s): 2.0
[163] Improvement: 33.767528876662254	Time (s): 2.013
[164] Improvement: 28.466753646731377	Time (s): 2.217
[165] Improvement: 24.095846578478813	Time (s): 2.145
[166] Improvement: 20.50084213912487	Time (s): 2.248
[167] Improvement: 17.546802565455437	Time (s): 2.511
[168] Improvement: 15.11777876317501	Time (s): 2.511
[169] Improvement: 13.116219311952591	Time (s): 2.556
[170] Improvement: 11.461442783474922	Time (s): 2.581
[171] Improvement: 10.087551802396774	Time (s): 2.581
[172] Improvement: 8.941186621785164	Time (s): 2.65
[173] Improvement: 7.9793776869773865	Time (s): 2.642
[174] Improvement: 7.16763973236084	Time (s): 2.722
[175] Improvement: 6.478336751461029	Time 

[310] Improvement: 0.5562622249126434	Time (s): 2.277
[311] Improvement: 0.5563607662916183	Time (s): 2.292
[312] Improvement: 0.5565235018730164	Time (s): 2.145
[313] Improvement: 0.5567473769187927	Time (s): 2.187
[314] Improvement: 0.5570349991321564	Time (s): 2.289
[315] Improvement: 0.557384729385376	Time (s): 2.183
[316] Improvement: 0.5577975213527679	Time (s): 2.225
[317] Improvement: 0.5582743436098099	Time (s): 2.244
[318] Improvement: 0.5588139295578003	Time (s): 2.208
[319] Improvement: 0.5594174116849899	Time (s): 2.227
[320] Improvement: 0.5600849837064743	Time (s): 1.985
[321] Improvement: 0.5608172565698624	Time (s): 2.116
[322] Improvement: 0.561614140868187	Time (s): 2.059
[323] Improvement: 0.5624764561653137	Time (s): 2.135
[324] Improvement: 0.5634038597345352	Time (s): 1.998
[325] Improvement: 0.5643981695175171	Time (s): 2.045
[326] Improvement: 0.5654593110084534	Time (s): 2.182
[327] Improvement: 0.5665865391492844	Time (s): 2.212
[328] Improvement: 0.567782908

[463] Improvement: 2.407333627343178	Time (s): 2.082
[464] Improvement: 2.46581894159317	Time (s): 2.012
[465] Improvement: 2.5271037369966507	Time (s): 2.544
[466] Improvement: 2.5913877487182617	Time (s): 2.254
[467] Improvement: 2.658891499042511	Time (s): 2.279
[468] Improvement: 2.7298512905836105	Time (s): 1.961
[469] Improvement: 2.804524302482605	Time (s): 1.987
[470] Improvement: 2.8831889778375626	Time (s): 2.387
[471] Improvement: 2.966150715947151	Time (s): 1.984
[472] Improvement: 3.053736090660095	Time (s): 1.947
[473] Improvement: 3.146306410431862	Time (s): 2.083
[474] Improvement: 3.244249552488327	Time (s): 2.066
[475] Improvement: 3.3479903787374496	Time (s): 2.071
[476] Improvement: 3.457990139722824	Time (s): 2.051
[477] Improvement: 3.5747537314891815	Time (s): 2.36
[478] Improvement: 3.6988263577222824	Time (s): 2.076
[479] Improvement: 3.83080692589283	Time (s): 2.04
[480] Improvement: 3.9713441282510757	Time (s): 2.087
[481] Improvement: 4.121147394180298	Time 

[617] Improvement: 5.665886670351028	Time (s): 2.147
[618] Improvement: 5.619381815195084	Time (s): 2.274
[619] Improvement: 5.573790267109871	Time (s): 2.079
[620] Improvement: 5.529041960835457	Time (s): 2.07
[621] Improvement: 5.485089689493179	Time (s): 2.058
[622] Improvement: 5.44191275537014	Time (s): 2.087
[623] Improvement: 5.399513140320778	Time (s): 2.034
[624] Improvement: 5.35791140794754	Time (s): 2.229
[625] Improvement: 5.317148655653	Time (s): 1.975
[626] Improvement: 5.277283847332001	Time (s): 2.102
[627] Improvement: 5.238384664058685	Time (s): 2.107
[628] Improvement: 5.2005340456962585	Time (s): 1.984
[629] Improvement: 5.163822203874588	Time (s): 2.063
[630] Improvement: 5.128344848752022	Time (s): 2.007
[631] Improvement: 5.094201892614365	Time (s): 1.91
[632] Improvement: 5.06149660050869	Time (s): 1.957
[633] Improvement: 5.030330389738083	Time (s): 2.013
[634] Improvement: 5.000803366303444	Time (s): 2.396
[635] Improvement: 4.973017111420631	Time (s): 1.937


[772] Improvement: 89.67437157034874	Time (s): 1.925
[773] Improvement: 72.94396761059761	Time (s): 2.171
[774] Improvement: 59.26004804670811	Time (s): 1.994
[775] Improvement: 48.14210344851017	Time (s): 2.185
[776] Improvement: 39.14132699370384	Time (s): 1.979
[777] Improvement: 31.88197973370552	Time (s): 2.16
[778] Improvement: 26.04896242916584	Time (s): 1.978
[779] Improvement: 21.37303240597248	Time (s): 2.063
[780] Improvement: 17.625661700963974	Time (s): 2.004
[781] Improvement: 14.617224916815758	Time (s): 1.837
[782] Improvement: 12.193961188197136	Time (s): 1.867
[783] Improvement: 10.233311846852303	Time (s): 2.181
[784] Improvement: 8.638650625944138	Time (s): 2.0
[785] Improvement: 7.334250673651695	Time (s): 2.108
[786] Improvement: 6.26090295612812	Time (s): 2.128
[787] Improvement: 5.372316658496857	Time (s): 1.961
[788] Improvement: 4.632232293486595	Time (s): 1.887
[789] Improvement: 4.012163937091827	Time (s): 2.113
[790] Improvement: 3.489649996161461	Time (s):

In [23]:
silence_model.freeze()

In [109]:
# write_model(root + '/models/bg_model', bg_model)
# write_model(root + '/models/silence_model', silence_model)

In [21]:
bg_model = read_gmm(root + '/models/bg_model')
silence_model = read_gmm(root + '/models/silence_model')

## Clean sentences, build sentence hmms

In [24]:
def wrap_with_silence(phones):
    return ['_'] + phones + ['_']

def remove_stress_combinations(phones):
    def remove_single_phone(phone):
        if phone[-1].isdigit():
            return phone[:-1]
        else:
            return phone
        
    return [remove_single_phone(phone) for phone in phones]

# example_phones = example.phones.loc[0].split(' ')
# example_phones_cleaned = wrap_with_silence(remove_stress_combinations(example_phones))
# example_phones_unique = np.unique(example_phones_cleaned)

# print(example_phones)
# print(example_phones_cleaned)
# print(example_phones_unique)

In [25]:
UNIQUE_PHONES = [
    'AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 
    'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 
    'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 
    'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH'
]

In [26]:
def make_phone_triple(bg_model):
    return tuple(bg_model.copy() for _ in range(3))

phone_distribution_triples = {
    phone: make_phone_triple(bg_model) for phone in UNIQUE_PHONES
}

In [27]:
def make_sentence_hmm(phones_cell, phone_triples, silence_model):
    phones = phones_cell.split(' ')
    cleaned_phones = wrap_with_silence(remove_stress_combinations(phones))
    hmm = pg.HiddenMarkovModel('-'.join(cleaned_phones))
    
    states = []
    for i, phone in enumerate(cleaned_phones):
        if phone == '_':
            phone_state = pg.State(silence_model.copy(), name='{}-SILENCE'.format(i))
            states.append(phone_state)
        else:
            phone_triple = phone_triples[phone]
            phone_states = (
                pg.State(phone_triple[0], name='{:02}a-{}'.format(i, phone)),
                pg.State(phone_triple[1], name='{:02}b-{}'.format(i, phone)),
                pg.State(phone_triple[2], name='{:02}c-{}'.format(i, phone))
            )
            states.extend(phone_states)
    hmm.add_states(states)
    
    hmm.add_transition(hmm.start, states[0], 1.0)
    for state in states:
        hmm.add_transition(state, state, 0.9)
    for prev, then in zip(states, states[1:]):
        hmm.add_transition(prev, then, 0.1)
    hmm.add_transition(states[-1], hmm.end, 0.1)
    hmm.bake()
    
    return hmm

# make_sentence_hmm(
#     script[script.sentence_id == 31].phones.iloc[0], 
#     phone_distribution_triples, silence_model
# );

In [28]:
recording_regex = re.compile(r'([mf])(\d+)/(\d+)_[mf]\d+_(\d+)')
def parse_recording(value):
    match = recording_regex.match(value)
    if match:
        return match.groups()
    
parse_recording('f0002/20150216173043137_f0002_31.pcm')

('f', '0002', '20150216173043137', '31')

In [29]:
unique_sentence_ids = np.sort(np.unique([int(parse_recording(path)[3]) for path in untrialed_paths]))
unique_sentence_ids_scripted = [sentence_id for sentence_id in unique_sentence_ids
                                if (script.sentence_id == sentence_id).any()
                                if isinstance(script[script.sentence_id == sentence_id].phones.iloc[0], str)]
print(len(unique_sentence_ids), len(unique_sentence_ids_scripted))

811 769


In [30]:
sentence_hmms = {
    sentence_id: make_sentence_hmm(
        script[script.sentence_id == sentence_id].phones.iloc[0], phone_distribution_triples, silence_model
    ) for sentence_id in unique_sentence_ids_scripted
}

In [31]:
print(len(sentence_hmms))

769


In [32]:
sentence_sequencess = {
    sentence_id: [all_mfcc[path] for path in untrialed_paths 
                                 if int(parse_recording(path)[3]) == sentence_id
                                 if all_mfcc[path].shape[0] >= len(script[script.sentence_id == sentence_id].phones.iloc[0].split(' ') * 3)]
    for sentence_id in unique_sentence_ids_scripted
}

## Train HMMs

In [33]:
def update_triples(phone_triples):
    empty = 0
    lost = 0
    for phone_triple in phone_triples.values():
        for gmm in phone_triple:
            if gmm.summaries.sum() == 0:
                empty += 1
                continue
            try:
                gmm.from_summaries()
            except Exception:
                gmm.clear_summaries()
                lost += 1
    return empty, lost

def train_epoch(sentence_sequencess, unique_sentence_ids, sentence_hmms, phone_triples):
    logps = []
    
    for sentence_id in unique_sentence_ids:
        print(sentence_id, end=' ')
        
        sentence_hmm = sentence_hmms[sentence_id]
        sentence_sequences = sentence_sequencess[sentence_id]
        weights = np.ones(len(sentence_sequences))
        
        logp = sentence_hmm.summarize(sequences=sentence_sequences, weights=weights, check_input=False)
                                      # algorithm='viterbi', 
        logps.append(logp)
    
    update_triples(phone_triples)
            
    return logps

In [34]:
write_triples(root + '/models/triples-init', phone_distribution_triples)

FileNotFoundError: [Errno 2] No such file or directory: '../input/reddots_r2015q4_v1/models/triples-init/AW-a'

In [ ]:
enrollments_4_td.phones[~enrollments_4_td.phones.isna()].apply(lambda row: len(row.split(' '))).describe()

In [ ]:
for i in range(10):
    for j in range(10):
        logps = train_epoch(sentence_sequencess, unique_sentence_ids_scripted, 
                            sentence_hmms, phone_distribution_triples)
        print('{}-{} logps'.format(i, j), sum(logps))
    path = root + '/models/triples-{}'.format(i)
    os.mkdir(path)
    write_triples(path, phone_distribution_triples)

In [160]:
update_triples(phone_distribution_triples)

(0, 0)

In [243]:
write_triples(root + '/models/triples', phone_distribution_triples)

In [21]:
phone_distribution_triples = read_triples(root + '/models/triples-1', UNIQUE_PHONES)

OSError: Cannot read model AW-a

In [105]:
print(example_sentence_hmm.states[3].distribution.distributions[0].mu)
print(example_sentence_hmm.states[10].distribution.distributions[0].mu)
print(example_sentence_hmm.states[10].name)

[  1.41467275e+01  -7.35344053e+00  -5.56301231e+00  -4.84370700e+00
  -3.26590152e+00  -4.67713110e+00  -2.93247692e+00  -6.21254481e+00
  -4.60166262e+00  -4.32767984e+00  -1.62883341e+00  -2.50476327e+00
  -8.24130769e+00   4.71618683e-01   4.57870227e-01  -1.32589895e+00
  -9.23610116e-01  -9.11397488e-01  -4.48633292e-02   6.83980317e-01
  -3.91819699e-01  -9.57840195e-02  -5.51740595e-01  -4.66018632e-01
  -3.27380013e-01   2.68423365e-03   6.40123001e-03   1.34323619e-01
   6.15838646e-03  -4.11852850e-02   7.89280213e-02  -2.26739330e-02
   5.75374672e-02   6.40953838e-03  -3.25918412e-02   1.42842274e-01
  -1.47236299e-01  -6.17011349e-02  -1.74521876e-02]
[  1.43344790e+01   1.12123728e-01  -4.61673784e+00  -4.37537285e+00
  -3.35688364e+00  -5.19159258e+00  -2.46247097e+00  -5.27950688e+00
  -6.93267329e+00   8.92074965e-01  -1.50063469e+00  -3.92398961e+00
  -1.10409868e+01   2.92374934e-01   1.52636161e+00  -1.38141933e+00
  -6.50400994e-01  -7.84072233e-01   1.30328483e-0

In [71]:
for i in (0, 50, 100):
    predictions = example_sentence_hmm.predict(example_features[i], algorithm='viterbi')
    print(i)
    print('\t'.join(example_sentence_hmm.states[prediction].name for prediction in predictions))

0
_-M-AY-_-V-OY-S-_-IH-Z-_-M-AY-_-P-AE-S-W-ER-D-_-start	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	0-SILENCE	1-M-beg	1-M-mid	1-M-end	2-AY-beg	2-AY-mid	2-AY-end	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-SILENCE	3-S

In [76]:
example_sentence_hmm.fit(example_features, algorithm='viterbi', n_jobs=12, verbose=True)

TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._from_summaries'
Traceback (most recent call last):
  File "pomegranate/bayes.pyx", line 710, in pomegranate.bayes.BayesModel.from_summaries
  File "pomegranate/distributions.pyx", line 2259, in pomegranate.distributions.MultivariateGaussianDistribution.from_summaries
TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'


TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._from_summaries'
Traceback (most recent call last):
  File "pomegranate/bayes.pyx", line 710, in pomegranate.bayes.BayesModel.from_summaries
  File "pomegranate/distributions.pyx", line 2259, in pomegranate.distributions.MultivariateGaussianDistribution.from_summaries
TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'


[1] Improvement: 4.373646683990955	Time (s): 522.3


...

[61] Improvement: 2.1627172827720642e-05	Time (s): 537.9


TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._from_summaries'
Traceback (most recent call last):
  File "pomegranate/bayes.pyx", line 710, in pomegranate.bayes.BayesModel.from_summaries
  File "pomegranate/distributions.pyx", line 2259, in pomegranate.distributions.MultivariateGaussianDistribution.from_summaries
TypeError: Cannot cast ufunc subtract output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'


[62] Improvement: -1.4025717973709106e-06	Time (s): 544.4
Total Training Improvement: 2551.5645321204793
Total Training Time (s): 33826.2032


2551.5645321204793

## Enroll

In [54]:
def make_speaker_classifier(inspeaker_classifier):
    n_components = 3 * len(phones)
    initial, transitions = make_matrix_beads(n_components)

    classifier = hmm.GMMHMM(
        n_components=n_components, n_mix=8, covariance_type='diag', init_params='mcw', params='tmcw'
    )

    classifier.startprob_ = initial
    classifier.transmat_ = transitions

GMMHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
    init_params='stmcw', n_components=39, n_iter=10, n_mix=8,
    params='stmcw', random_state=None, startprob_prior=1.0, tol=0.01,
    transmat_prior=1.0, verbose=False)

In [ ]:
def perform_enrollments(classifier, enrollments):
    labels = enrollments[['is_male', 'speaker_id', 'sentence_id']].values
    features = [load_mfcc(path) for path in enrollments_1['pcm_path']]
    classifier.fit(features, labels)
    

## Trial

In [ ]:
def save_results(label, results):
    path = os.path.join(root, 'result', label)
    with open(path) as opened:
        pickle.dump(results, opened)
        
def load_results(label):
    path = os.path.join(root, 'result', label)
    with open(path) as opened:
        return pickle.load(opened)
    
def perform_trial(classifier, path):
    features = load_mfcc(path)
    return classifier.predict_single_proba(features)

def perform_trials(classifier, trials):
    paths = trials['pcm_path'].unique()
    results = {}
    for path in paths:
        results[path] = perform_trial(classifier, path)
    return results

In [10]:
per_phone_weights = {}

per_phone_weights.get('A', np.eye(3))

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [ ]:
class Bakis:
    def __init__(self, phones, per_phone_weights, ignore_stress=True, wrap_silence=True):
        self.phone_seq = phones.split(' ')
        if wrap_silence:
            self.phone_seq = ['_'] + self.phone_seq + ['_']
            
        state_transitions = 

In [51]:
enrollments_1.sentence_id.unique()

array([31, 32, 33, 34, 35, 36, 37, 38, 39, 40])

In [52]:
sample_enrollments = enrollments_1.loc[enrollments_1.sentence_id == 31]

In [ ]:
class HmmGmmVerifier:
    def __init__(self):
        self.patterns = None
        self.labels = None

    def fit(self, features, labels):
        self.patterns = features
        self.labels = labels
        self.unique_labels = np.unique(labels)
        
    def enroll(self, ):
        pass
    
    def trial(self, ):
        pass

    def predict(self, features, metric=spatial.distance.cosine):
        sequence_label_proba = self.predict_proba(features, metric)
        max_proba_index = sequence_label_proba.argmax(axis=1)
        return self.unique_labels[max_proba_index]
    
    def predict_proba(self, features, metric=spatial.distance.cosine):
        sequence_n = len(features) 
        pattern_n = len(self.patterns)
        
        sequence_label_proba = np.zeros((sequence_n, pattern_n), dtype=self.labels.dtype)
        for i, sequence in enumerate(features):
            sequence_label_proba[i, :] = self.predict_single_proba(sequence, metric)
            
        return sequence_label_proba
    
    def predict_single_proba(self, sequence, metric=spatial.distance.cosine):
        pattern_dists = np.zeros(len(self.patterns), dtype=np.float64)
        for i, pattern in enumerate(self.patterns):
            distance, _ = fastdtw.fastdtw(pattern, sequence, dist=metric)
            pattern_dists[i] = distance
            
        pattern_proba = np.exp(-pattern_dists)
        
        label_proba = np.zeros(len(self.unique_labels), dtype=np.float64)
        all_dim = tuple(range(1, self.labels.ndim))
        for i, label in enumerate(self.unique_labels):
            relevant = (self.labels == label).all(axis=all_dim)
            total_proba = pattern_proba[relevant].sum()
            label_proba[i] = total_proba
        
        return label_proba / label_proba.sum()

classifier = markov.HMMGMMClassifier()
classifier.fit(e_features, e_labels, n_components=10, n_mix=2)
return classifier.predict(t_features)

In [5]:
display(enrollments_1.groupby(['is_male', 'speaker_id']).size())

is_male  speaker_id
False    2             30
         4             30
         5             30
         6             30
         8             24
         12            30
True     1             30
         2             30
         4             30
         5             30
         6             30
         7             30
         8              6
         9             30
         13            30
         14            24
         15            30
         16            30
         17            30
         18            30
         19            30
         20            30
         21            24
         22            30
         23            30
         26            30
         28            30
         29            30
         32            30
         38            24
         40            30
         41            30
         43            30
         47            30
         48             6
         51            30
         52             6
         53       